<a href="https://colab.research.google.com/github/violjii/lab_Serhiienko/blob/main/Lab3_Serhiienko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# !pip install torch torchvision

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

df = pd.read_csv('data.csv')
print(df.head())
print(df.info())
print(df.describe())
print(df.isnull().sum())

X = df.drop('cured', axis=1).values
y = df['cured'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))

model = LogisticRegressionModel(X_train.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

epochs = 100
losses = []

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train_tensor)
    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

plt.plot(losses)
plt.title("Функція втрат під час навчання (PyTorch)")
plt.xlabel("Епоха")
plt.ylabel("Втрати")
plt.show()

model.eval()
with torch.no_grad():
    y_pred_prob = model(X_test_tensor)
    y_pred = (y_pred_prob > 0.5).float()

acc = (y_pred.eq(y_test_tensor).sum().item()) / len(y_test_tensor)
print(f"🎯 Точність моделі (PyTorch): {acc:.4f}")
print("🔢 Фінальна функція втрат:", losses[-1])

clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred_sklearn = clf.predict(X_test)

print("\n🤖 Метрики sklearn логістичної регресії:")
print("Accuracy:", accuracy_score(y_test, y_pred_sklearn))
print("Classification Report:\n", classification_report(y_test, y_pred_sklearn))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_sklearn))

y_pred_np = y_pred.numpy().flatten()
print("\n🧠 Метрики PyTorch моделі:")
print("Classification Report:\n", classification_report(y_test, y_pred_np))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_np))

print("""
📌 Висновки:
- Обидві моделі показали подібну точність.
- PyTorch дозволяє реалізувати логістичну регресію на нейронному рівні та масштабувати її.
- sklearn LogisticRegression швидша у використанні для простих задач.
- Обидві моделі демонструють високі метрики, але PyTorch надає більше гнучкості для подальших модифікацій.
""")


   Phoenix Feather  Unicorn Horn  Dragon's Blood  Mermaid Tears  Fairy Dust  \
0              2.4          18.7            18.4           27.9         7.9   
1              2.1           6.0            15.0           13.3        15.6   
2             17.2          13.9            23.8            6.8        10.7   
3              8.4           9.7             6.8           26.9         4.6   
4             22.1          10.8            16.4           10.5        22.0   

   Goblin Toes  Witch's Brew  Griffin Claw  Troll Hair  Kraken Ink  \
0          9.6          18.3          13.2         2.5        26.0   
1         13.1          11.0           5.0         7.2        26.0   
2         15.8          19.4           2.7        15.4        21.2   
3         29.1          14.6          19.7        18.0        20.8   
4         23.4           2.6          18.2        23.8        11.3   

   Minotaur Horn  Basilisk Scale  Chimera Fang  Cured  
0           10.5            26.2          12.5  

KeyError: "['cured'] not found in axis"